## Практическое задание к уроку № 10 по теме "Распознавание лиц и эмоций".

*Нужно написать приложение, которое будет считывать и выводить кадры с  
веб-камеры. В процессе считывания определять что перед камерой находится  
человек, задетектировав его лицо на кадре. После этого, человек показывает жесты  
руками, а алгоритм должен считать их и определенным образом реагировать на эти  
жесты.*

В данном задании будем использовать библиотеку [mediapipe](https://mediapipe.dev/) для детектирования  
лиц и жестов. Суть задания в том, чтобы сначала детектировать лицо пользователя,  
а потом детектировать и классифицировать показанный им жест. В mediapipe детектирование  
жестов происходит в 2 этапа - сначала непосредственно детектирование, а затем  
обнаружение особых точек (landmarks) жеста. Фреймворк выдаёт результат уже второго  
этапа, поэтому мы будем работать с этими особыми точками и их координатами, а не  
bounding box'ами детектированных жестов. Из-за этого в качестве классификатора  
у нас будет обычная логистическая регрессия, а не свёрточная нейронная сеть.

Загрузим необходимые библиотеки:

In [1]:
import cv2
import glob
from google.protobuf.json_format import MessageToDict
import mediapipe as mp
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands

В качестве датасета для распознавания жестов используем один из  
датасетов kaggle. (https://www.kaggle.com/datasets/gti-upm/leapgestrecog)  
В этом датасете 10 разных людей показывают 10 жестов. Из этих 10 жестов  
мы выберем 3, на которые будет реагировать модель. Также возьмём еще 3 жеста,  
которые будут принадлежать к классу *разное*, чтобы модель понимала, что кроме  
нужных нам жестов, пользователь может показывать и другие, а не причисляла  
любой жест к одному из трёх целевых.

Получим необходимые имена файлов, относящиеся к 6 классам:

In [3]:
files = glob.glob('./leapGestRecog/*/0[13457]*/*') + glob.glob('./leapGestRecog/*/10*/*')

Напишем функцию, которая на вход будет принимать особые точки жеста,  
а возвращать их масштабированные координаты. При формировании обучающего  
датафрейма будем ещё добавлять таргет:

In [4]:
def landmarks_to_df(landmarks, df=None, target=None):
    
    # Переводим в привычный словарь
    landmarks = MessageToDict(landmarks[0])['landmark']
    
    # Заполняем списки с координатами
    x = np.empty(len(landmarks))
    y = np.empty(len(landmarks))
    z = np.empty(len(landmarks))
    
    for i, v in enumerate(landmarks):
        x[i] = v['x']
        y[i] = v['y']
        z[i] = v['z']
    
    # Масшабируем координаты, чтобы не имело значения, в какой
    # области кадра наш жест
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x.reshape(-1, 1)).reshape(1, -1)
    y = scaler.fit_transform(y.reshape(-1, 1)).reshape(1, -1)
    z = scaler.fit_transform(z.reshape(-1, 1)).reshape(1, -1)
    
    # Подготавливаем и возвращаем датасет
    if target is not None:
        features = np.c_[x, y, z, target]
    else:
        features = np.c_[x, y, z]
    
    if df is not None:
        df = pd.concat((df, pd.DataFrame(features)), axis=0)
    else:
        df = pd.DataFrame(features)
    
    return df

Чтобы внести координаты особых точек в датафрейм, сначала нужно найти  
эти точки. Для этого прогоняем все обучающие картинки через фреймворк,  
а затем вносим их обработанные координаты в датафрейм. Также присваиваем  
значения таргета жестам, где 3 целевых жеста получают свой класс, а  
остальные 3 жеста получают нулевой класс. Картинки, где фреймфорк не  
обнаружил жест, пропускаются.

In [16]:
mapping_dict = {'03': 1, '07': 2, '10': 3}
df = pd.DataFrame()

with mp_hands.Hands(max_num_hands=1, static_image_mode=True) as hands:
    
        for idx, file in enumerate(files):
            image = cv2.imread(file)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not results.multi_hand_landmarks:
                continue
            
            target = mapping_dict.get(re.findall('frame_\d{2}_(\d{2})', file)[0], 0)
            df = landmarks_to_df(results.multi_hand_landmarks, df, target)

Посмотрим на получившийся датафрейм:

In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000000,0.311541,0.512761,0.681499,0.850143,0.422335,0.632079,0.779927,0.878924,0.357095,...,0.706946,0.616383,0.509567,0.362676,0.263579,0.406593,0.224233,0.091495,0.00000,2.0
0,0.048491,0.000000,0.065394,0.132109,0.145065,0.358238,0.431868,0.447997,0.413534,0.451846,...,0.945124,0.995335,0.934495,0.892546,0.888900,0.947669,0.932828,0.914559,0.90442,2.0
0,0.000000,0.338170,0.574525,0.754217,0.924684,0.471106,0.711755,0.867982,0.969665,0.390790,...,0.666939,0.590660,0.502993,0.345904,0.228920,0.359374,0.198525,0.084720,0.00000,2.0
0,0.000000,0.295899,0.508655,0.670603,0.823677,0.426636,0.641877,0.784661,0.879639,0.368422,...,0.679757,0.536571,0.455862,0.334093,0.245186,0.299947,0.160995,0.069313,0.00000,2.0
0,0.000000,0.324373,0.567884,0.759133,0.927048,0.475913,0.716733,0.868789,0.969522,0.392026,...,0.600458,0.593749,0.476638,0.297750,0.169646,0.369482,0.191877,0.080038,0.00000,2.0


In [18]:
df.shape

(5878, 64)

In [20]:
df[63].value_counts()

0.0    2579
2.0    1610
3.0    1169
1.0     520
Name: 63, dtype: int64

Баланс классов удовлетворительный, сохраним датасет:

In [19]:
df.to_csv('./data.csv', index=False)

Ячейка для загрузки датасета из файла:

In [5]:
# df = pd.read_csv('./data.csv')
# df.columns = df.columns.astype(int)

Разделим данные на обучающую и тестовую выборки, сделаем  
стратификацию по таргету:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, stratify=df[63], random_state=29)

Обучим логистическую регрессию:

In [7]:
model = LogisticRegression(max_iter=400)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Посмотрим на метрики:

In [8]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       516
         1.0       0.94      0.95      0.95       104
         2.0       0.99      1.00      1.00       322
         3.0       0.99      0.97      0.98       234

    accuracy                           0.98      1176
   macro avg       0.98      0.98      0.98      1176
weighted avg       0.98      0.98      0.98      1176



Модель очень хорошо научилась разделять классы.  
Перед использованием заново обучим её на всём датасете:

In [9]:
model.fit(df.iloc[:, :-1], df.iloc[:, -1]);

Теперь у нас есть всё для работы программы. Будем брать изображение  
с веб-камеры и детектировать на нём лицо. Если лицо не обнаружено, то  
выводится соответствующее сообщение. Если обнаружено, то производится  
поиск жеста. При обнаружении жеста, координаты особых точек передаются  
модели классификации, и, в зависимости от жеста, на экран выводится  
соответствующее сообщение.

In [11]:
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection() as face_detection:
    with mp_hands.Hands(max_num_hands=1) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print('Ignoring empty camera frame.')
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)
            
            # Несмотря на наличие параметра min_detection_confidence у FaceDetection(), он,  
            # похоже, не работает. Поэтому зададим порог уверенности вручную (0,85), чтобы  
            # избежать ложных срабатываний детектирования лица
            if results.detections and MessageToDict(results.detections[0])['score'][0] > 0.85:
                
                results = hands.process(image)
                
                if results.multi_hand_landmarks:
                    gesture = model.predict_proba(landmarks_to_df(results.multi_hand_landmarks))[0]
                    
                    if max(gesture) > 0.5:
                        gesture = np.argmax(gesture)
                        
                        match gesture:
                            case 0:
                                text = 'Unknown sign!'
                            case 1:
                                text = 'Hello!'
                            case 2:
                                text = 'OK!'
                            case 3:
                                text = 'Scary!'
                    
                    # Если модель не уверена во всех классах, но фреймворк всё равно находит жест,
                    # на экран будет выводиться сообщение
                    else:
                        text = 'Is it a sign?'
            
                
                    cv2.putText(image, text, (170, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                1.3, (0, 0, 255), 2)
            
            else:
                cv2.putText(image, 'Is anyone here?', (170, 50),
                            cv2.FONT_HERSHEY_COMPLEX,
                            1.3, (0, 0, 255), 2)
            
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()
cv2.destroyAllWindows()